In [ ]:
from pylab import *
import h5py
from numpy import *
f = h5py.File("data/GreenlandInBedCoord.h5") #Change the path to your local location or try:
#f = h5py.File("https://katie.mtech.edu/classes/csci444/python/data/GreenlandInBedCoord.h5")
H  = array(f["thickness"],dtype=float32) # Ice thickness (m)
ux = array(f["VX"],dtype=float32)        # x component of velocity (m/year)
uy = array(f["VY"],dtype=float32)        # y component of velocity (m/year)
S  = array(f["surface"],dtype=float32)   # Surface elevation (m above sea level)
B  = array(f["bed"],dtype=float32)       # Elevation of material under ice (m above sea level)
T  = array(f["t2m"],dtype=float32)      # Mean annual temperature at 2 m above surface (C)
A  = array(f["smb"],dtype=float32)       # Rate of ice accumulation (m/year)
X  = array(f["x"])                       # x coordinate of data
Y  = array(f["y"])                       # y coordinate of data


# Standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import plotly.figure_factory as ff

# Data science imports
import numpy as np

#Reduce the number of data points by reduction factor
reduce = 50

#Create a grid of points for the velocities
x,y = np.meshgrid(X[::reduce], Y[::reduce])
u = ux[::reduce,::reduce]
v = uy[::reduce,::reduce]
print(x.shape, y.shape, u.shape, v.shape)

reduce = 10

S = S[::reduce,::reduce]


In [ ]:
#Seems to be needed to get offline plot to display
import cufflinks
cufflinks.go_offline()

#Create the arrows, use scale to increase arrow size
#A figure is returned, with a scattered data component
fig = ff.create_quiver(x, y, u, v,
                       scale=1000,
                       arrow_scale=.3,
                       name='quiver',
                       line=dict(width=2,color='green'))

#Add the figure layout
layout = go.Layout(
    width=500,
    height=700,
    autosize=False,
    margin=dict(t=50, b=50, l=50, r=50)
)

fig['layout'] = layout
iplot(fig, filename='greenland_quiver')


# Next combine the quiver with the glacier surface data
Note that the velocities are scaled by 1000, so the arrows represent how far the point would move in a 1000 years if the same velocity is maintained.

In [ ]:
fig['data']

In [ ]:
layout = go.Layout(
    width=800,
    height=800,
    autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230, 230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        aspectratio = dict(x=1, y=1, z=0.7),
        aspectmode = 'manual'
    )
)

updatemenus=list([
    dict(
        buttons=list([   
            dict(
                args=['type', 'surface'],
                label='Quiver Wind',
                method='restyle'
            ),
            dict(
                args=['type', 'heatmap'],
                label='Glacier Height',
                method='restyle'
            )             
        ]),
        #button properties
        direction = 'down',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.1,
        xanchor = 'left',
        y = 1.1,
        yanchor = 'top' 
    ),
])

layout['updatemenus'] = updatemenus

trace2 = go.Heatmap(z=S)

data = [fig['data'][0], trace2]

fig = go.Figure(data=data,layout=layout)

iplot(fig, filename='glaciertime')